In [1]:
from fuvtds_base_class import FUVTDSBase

In [2]:
monitor = FUVTDSBase(PIDs='fuvtds_analysis_list.dat')

  0%|          | 0/1899 [00:00<?, ?it/s]

[array('/grp/hst/cos2/cosmo/15535/ldv004u6q_x1d.fits.gz', dtype='<U47'), array('/grp/hst/cos2/cosmo/15535/ldv057z1q_x1d.fits.gz', dtype='<U47'), array('/grp/hst/cos2/cosmo/15535/ldv009axq_x1d.fits.gz', dtype='<U47'), array('/grp/hst/cos2/cosmo/15535/ldv011jtq_x1d.fits.gz', dtype='<U47'), array('/grp/hst/cos2/cosmo/15773/le5g02bzq_x1d.fits.gz', dtype='<U47'), array('/grp/hst/cos2/cosmo/15773/le5g54jtq_x1d.fits.gz', dtype='<U47'), array('/grp/hst/cos2/cosmo/15773/le5g06cjq_x1d.fits.gz', dtype='<U47'), array('/grp/hst/cos2/cosmo/15773/le5g07mvq_x1d.fits.gz', dtype='<U47'), array('/grp/hst/cos2/cosmo/15773/le5g09g9q_x1d.fits.gz', dtype='<U47'), array('/grp/hst/cos2/cosmo/15773/le5g11k6q_x1d.fits.gz', dtype='<U47'), array('/grp/hst/cos2/cosmo/16324/lefe02kjq_x1d.fits.gz', dtype='<U47'), array('/grp/hst/cos2/cosmo/16324/lefe04otq_x1d.fits.gz', dtype='<U47'), array('/grp/hst/cos2/cosmo/16324/lefe06esq_x1d.fits.gz', dtype='<U47'), array('/grp/hst/cos2/cosmo/16324/lefe07lrq_x1d.fits.gz', dtype=

KeyboardInterrupt: 

In [ ]:
ls = [0,1,2,3,4]
a = {
    'a': ls
}

print(a['a'])

[0, 1, 2, 3, 4]


In [ ]:
a['a'][4] = 7
print(a['a'])

[0, 1, 2, 3, 7]


In [ ]:
print(ls)

[0, 1, 2, 3, 7]


In [ ]:
file = '/user/jhernandez/nuvtds_2024/scientific_testing/data/old/17395/lf4v01xhq_x1d.fits'

In [ ]:
from astropy.io import fits

In [ ]:
data = fits.getdata(file, 1)

In [ ]:
for segment in data['SEGMENT']:
    print(segment)

NUVA
NUVB
NUVC
